#### **Ingesta del archivo "country.json"**

In [0]:
dbutils.widgets.text("environment","")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-16")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Librerías

In [0]:
from pyspark.sql.functions import lit,current_timestamp

##### 1. Lectura del archivo JSON

##### Esquema

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

##### Leer JSON

In [0]:
df_countries = spark.read.schema(countries_schema).json(f"{bronze_folder_path}/{var_file_date}/country.json")

#### Eliminar columnas

In [0]:
df_countries_dropped = df_countries.drop(df_countries.countryIsoCode)

#### Agregar y renombrar columnas

In [0]:
df_final = add_ingestion_date(df_countries_dropped).withColumn("environment",lit(var_environment)) \
                               .withColumnsRenamed({"countryId":"country_id","countryName":"country_name"}).withColumn("file_date",lit(var_file_date))

#### Escribir dataframe en Silver

In [0]:
#df_final.write.mode("overwrite").parquet(f"{silver_folder_path}/countries")
df_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
dbutils.notebook.exit("Success")